In [1]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, balanced_accuracy_score
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, roc_curve, auc

Проведём предварительную обработку данных:

In [8]:
# Загрузим датасет logs_exp.csv с помощью библиотеки pandas в Jupyter-ноутбуке
logs_exp_data = pd.read_csv(r'C:\Users\Dima\Desktop\aaa_GUAP_\Боженко\logs_exp.csv', sep='\t')
orders_data = pd.read_csv(r'C:\Users\Dima\Desktop\aaa_GUAP_\Боженко\orders.csv')
visitors_data = pd.read_csv(r'C:\Users\Dima\Desktop\aaa_GUAP_\Боженко\visitors.csv')

Выведем первые 20 строк при помощи команды head:

In [7]:
logs_exp_data.head(20)

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
5,CartScreenAppear,6217807653094995999,1564055323,248
6,OffersScreenAppear,8351860793733343758,1564066242,246
7,MainScreenAppear,5682100281902512875,1564085677,246
8,MainScreenAppear,1850981295691852772,1564086702,247
9,MainScreenAppear,5407636962369102641,1564112112,246


In [10]:
orders_data.head(20)

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B
5,182168103,935554773,2019-08-15,2210,B
6,398296753,2900797465,2019-08-15,1860,B
7,2626614568,78758296,2019-08-15,1044,A
8,1576988021,295230930,2019-08-15,13710,A
9,1506739906,1882260405,2019-08-15,1855,B


In [13]:
visitors_data.head(20)

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756
5,2019-08-06,A,667
6,2019-08-07,A,644
7,2019-08-08,A,610
8,2019-08-09,A,617
9,2019-08-10,A,406


In [18]:
# Проверим датафреймы на наличие пропущеных значений
print(logs_exp_data.isna().sum(), orders_data.isna().sum(), visitors_data.isna().sum(), sep='\n\n')

EventName         0
DeviceIDHash      0
EventTimestamp    0
ExpId             0
dtype: int64

transactionId    0
visitorId        0
date             0
revenue          0
group            0
dtype: int64

date        0
group       0
visitors    0
dtype: int64


Преобразуем данные с датами к формату datetime:

In [20]:
orders_data['date'] = pd.to_datetime(orders_data['date'])
visitors_data['date'] = pd.to_datetime(visitors_data['date'])

In [21]:
# Посмотрим, есть ли пересекающие пользователи в таблице orders и удалим их
orders_data_A = orders_data[orders_data['group'] == 'A']
orders_data_B = orders_data[orders_data['group'] == 'B']

for indexA, rowA in orders_data_A.iterrows():
    for indexB, rowB in orders_data_B.iterrows():
        if rowA['visitorId'] == rowB['visitorId']:
            orders_data = orders_data.drop(orders_data[orders_data["visitorId"] == rowA['visitorId']].index)

In [23]:
# Создадим массив уникальных пар значений дат и групп теста и получим агрегированные кумулятивные данные по дням о заказах
datesGroups = orders_data[['date','group']].drop_duplicates()
ordersAK = datesGroups.apply(lambda x: orders_data[np.logical_and(orders_data['date'] <= x['date'], orders_data['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'transactionId' : 'nunique', 'visitorId' : 'nunique', 'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])
# Получим агрегированные кумулятивные данные по дням о посетителях
visitorsAK = datesGroups.apply(lambda x: visitors_data[np.logical_and(visitors_data['date'] <= x['date'], visitors_data['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'visitors' : 'sum'}), axis=1).sort_values(by=['date','group'])

Объединим полученные данные в одной таблице

In [24]:
cumulativeData = ordersAK.join(visitorsAK.drop(['date', 'group'], axis=1))
cumulativeData.rename(columns={'transactionId': 'orders', 'visitorId': 'byers'}, inplace=True)
cumulativeData.head(5)

,date,group,orders,byers,revenue,visitors
55,2019-08-01,A,23,19,142779,719
66,2019-08-01,B,17,17,59758,713
175,2019-08-02,A,42,36,234381,1338
173,2019-08-02,B,40,39,221801,1294
291,2019-08-03,A,66,60,346854,1845


In [25]:
# Разделим cumulativeData на cumulativeRevenueA и cumulativeRevenueB – для группы А и B
cumulativeRevenueA = cumulativeData[cumulativeData['group'] == 'A']
cumulativeRevenueB = cumulativeData[cumulativeData['group'] == 'B']

In [26]:
# Получим 90, 95, 99 перцентили числа заказов на пользователя

ordersByUsers = orders_data.groupby('visitorId')['transactionId'].count().sort_values(ascending = False).reset_index()
ordersByUsers.rename(columns={'transactionId': 'orders'}, inplace=True)

In [32]:
print(f"90 перцентиль: {np.percentile(ordersByUsers['orders'], 90)}\n95 перцентиль: \
{np.percentile(ordersByUsers['orders'], 95)}\n99 перцентиль: {np.percentile(ordersByUsers['orders'], 99)}")

90 перцентиль: 1.0
95 перцентиль: 1.0
99 перцентиль: 2.0


Вывод: 90 и 95 перцентили оказались равными 1, что означает, что 95% пользователей совершили не больше 1 заказа. Значение 99 перцентиля равное 2 означает, что 99% пользователей совершили не более двух заказов.